In [8]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
from topoPy import *

df = nc.Dataset('../data/icon_grid_0010_R02B04_G_linked.nc')

clat = df.variables['clat'][:]
clon = df.variables['clon'][:]
clat_verts = df.variables['clat_vertices'][:]
clon_verts = df.variables['clon_vertices'][:]
links = df.variables['links'][:]

# clat = clat*(180/np.pi)
# clon = clon*(180/np.pi)
# clat_verts = clat_verts*(180/np.pi)
# clon_verts = clon_verts*(180/np.pi)

datfile = '../data/GMTED2010_topoGlobal_SGS_30ArcSec.nc'
var = {'name':'topo','units':'m'}

np.random.seed(555)
# icon_cell_indexes = np.sort([440, 19442, 5595, 5026, 4793, 4631])
# icon_cell_indexes = np.random.randint(0,np.size(clat)-1,36)
icon_cell_indexes = np.array([  343,  1021,  1367,  2045,  2391,  3069,  3415,  4093,  4439,
        5117,  5588,  5603,  5985,  6012,  6612,  6627,  7009,  7036,
        7636,  7651,  8033,  8060,  8660,  8675,  9057,  9084,  9684,
        9699, 10081, 10108]) # cells that are not being found on the grid...

# icon_cell_indexes = [3027,3028,3029]
# Mount Ebrus; Firehorn; Taunus; Pirin; Langtang; ???
print(icon_cell_indexes)
print(icon_cell_indexes.shape)

[  343  1021  1367  2045  2391  3069  3415  4093  4439  5117  5588  5603
  5985  6012  6612  6627  7009  7036  7636  7651  8033  8060  8660  8675
  9057  9084  9684  9699 10081 10108]
(30,)


In [9]:
comp_clat = clat[icon_cell_indexes]
comp_clon = clon[icon_cell_indexes]
comp_clat_verts = clat_verts[icon_cell_indexes]
comp_clon_verts = clon_verts[icon_cell_indexes]

ncfile = Dataset('../data/icon_compact.nc',mode='w') 
print(ncfile)

cell = ncfile.createDimension('cell', np.size(comp_clat))     # latitude axis
nv = ncfile.createDimension('nv', 3)    # longitude axis
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact ICON grid for testing and debugging purposes'
print(ncfile.title)

clat = ncfile.createVariable('clat', np.float32, ('cell',))
clat.units = 'radian'
clat.long_name = 'center latitude'

clon = ncfile.createVariable('clon', np.float32, ('cell',))
clon.units = 'radian'
clon.long_name = 'center longitude'

clat_verts = ncfile.createVariable('clat_vertices', np.float32, ('cell','nv',))
clat_verts.units = 'radian'

clon_verts = ncfile.createVariable('clon_vertices', np.float32, ('cell','nv',))
clon_verts.units = 'radian'

clat[:] = comp_clat
clon[:] = comp_clon
clat_verts[:,:] = comp_clat_verts
clon_verts[:,:] = comp_clon_verts

print(clon_verts[:,:])

ncfile.close(); print('Dataset is closed!')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
('cell', <class 'netCDF4._netCDF4.Dimension'>: name = 'cell', size = 30)
('nv', <class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 3)
Compact ICON grid for testing and debugging purposes
[[ 0.03615208  0.03332213  0.06777534]
 [ 1.220485    1.1888617   1.2233149 ]
 [ 1.2927891   1.2899592   1.3244123 ]
 [ 2.477122    2.4454987   2.479952  ]
 [ 2.5494263   2.5465963   2.5810494 ]
 [-2.5494263  -2.5810494  -2.5465963 ]
 [-2.477122   -2.479952   -2.4454987 ]
 [-1.2927891  -1.3244123  -1.2899592 ]
 [-1.220485   -1.2233149  -1.1888617 ]
 [-0.03615208 -0.06777534 -0.03332213]
 [ 0.91149545  0.93568635  0.89572924]
 [ 0.81458193  0.791459    0.83178616]
 [ 0.4420551   0.42485094  0.465178  ]
 [ 0.3451416   0.3609078   0.3209507 ]
 [ 2.1681325   2.1923234   2.1523664 ]
 [ 2.071219    2.0480962   2.0884233 ]
 [ 1.6986922   1.681488    1.72

In [10]:
links_tmp = links[icon_cell_indexes].flatten()
links_tmp = links_tmp[np.where(links_tmp > 0)]
links_tmp = np.sort(list(set(links_tmp)))
links_tmp -= 1

print(links_tmp)

lon, lat, z = readnc(datfile, var)
nrecords = np.shape(z)[0]; nlon = np.shape(lon)[1]; nlat = np.shape(lat)[1]

sz_tmp = np.size(links_tmp)
# print(nlat,nlon, np.size(links_tmp))

compactified_topo = np.zeros((sz_tmp,nlat, nlon))
print(compactified_topo.shape)

compactified_lat = np.zeros((sz_tmp,nlat))
compactified_lon = np.zeros((sz_tmp,nlon))
    
for i,lnk in enumerate(links_tmp):
    print("i, lnk = ", (i, lnk))
    compactified_lat[i] = lat[lnk]
    compactified_lon[i] = lon[lnk]
    compactified_topo[i] = z[lnk]
    
del lat, lon, z

ncfile = Dataset('../data/topo_compact.nc',mode='w',format='NETCDF4_CLASSIC') 
print(ncfile)

nfiles = ncfile.createDimension('nfiles', sz_tmp)
lat = ncfile.createDimension('lat', nlat)
lon = ncfile.createDimension('lon', nlon)
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact GMTED2010 USGS Topography grid for testing and debugging purposes'
print(ncfile.title)

lat = ncfile.createVariable('lat', np.float32, ('nfiles','lat'))
lat.units = 'degrees'

lon = ncfile.createVariable('lon', np.float32, ('nfiles','lon'))
lon.units = 'degrees'

topo = ncfile.createVariable('topo', np.float32, ('nfiles','lat','lon'))
topo.units = 'meters'

lat[:,:] = compactified_lat
lon[:,:] = compactified_lon
topo[:,:,:] = compactified_topo

ncfile.close(); print('Dataset is closed!')

[ 0  2  3  6  7 10 11 14 15 18 19 22 24 26 27 30 31 34 35 38 39 42 43 46]
{'lat': <class 'netCDF4._netCDF4.Variable'>
float32 lat(nfiles, lat)
    units: degrees
unlimited dimensions: 
current shape = (108, 2400)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lon': <class 'netCDF4._netCDF4.Variable'>
float32 lon(nfiles, lon)
    units: degrees
unlimited dimensions: 
current shape = (108, 3600)
filling on, default _FillValue of 9.969209968386869e+36 used, 'topo': <class 'netCDF4._netCDF4.Variable'>
float32 topo(nfiles, lat, lon)
    units: meters
unlimited dimensions: 
current shape = (108, 2400, 3600)
filling on, default _FillValue of 9.969209968386869e+36 used}
(24, 2400, 3600)
i, lnk =  (0, 0)
i, lnk =  (1, 2)
i, lnk =  (2, 3)
i, lnk =  (3, 6)
i, lnk =  (4, 7)
i, lnk =  (5, 10)
i, lnk =  (6, 11)
i, lnk =  (7, 14)
i, lnk =  (8, 15)
i, lnk =  (9, 18)
i, lnk =  (10, 19)
i, lnk =  (11, 22)
i, lnk =  (12, 24)
i, lnk =  (13, 26)
i, lnk =  (14, 27)
i, lnk =  (15, 30)
i, lnk 

In [ ]:
lon, lat, z = readnc(datfile, var)
nrecords = np.shape(z)[0]; nlon = np.shape(lon)[1]; nlat = np.shape(lat)[1]

In [19]:
print(lon[1][:])
print(lon[2][:])
print(lon[3][:])
print(lon[4][:])

[4.1666669e-03 1.2500000e-02 2.0833334e-02 ... 2.9979166e+01 2.9987499e+01
 2.9995832e+01]
[30.004168 30.0125   30.020834 ... 59.979168 59.9875   59.995834]
[-2.9995832e+01 -2.9987499e+01 -2.9979166e+01 ... -2.0833334e-02
 -1.2500000e-02 -4.1666669e-03]
[30.004168 30.0125   30.020834 ... 59.979168 59.9875   59.995834]


In [20]:
print(lat[1][:])
print(lat[2][:])
print(lat[3][:])
print(lat[4][:])

[-9.995833 -9.9875   -9.979167 ...  9.979167  9.9875    9.995833]
[10.004167 10.0125   10.020833 ... 29.979166 29.9875   29.995832]
[10.004167 10.0125   10.020833 ... 29.979166 29.9875   29.995832]
[-9.995833 -9.9875   -9.979167 ...  9.979167  9.9875    9.995833]
